# Repeat from 4e1

In [1]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd

from sklearn.preprocessing import QuantileTransformer
from joblib import load

In [2]:
pipe = load("mypipe.joblib")

In [3]:
server = 'fs.rice.edu'
database = 'stocks'
username = 'stocks'
password = '6LAZH1'
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database 

conn = create_engine(string).connect()

In [4]:
df = pd.read_sql(
    """
    select ticker, bm, mom12m, roeq, siccd
    from ghz
    order by ticker
    """,
    conn
)
df = df.dropna()
df = df.set_index("ticker")
features = ["bm", "mom12m", "roeq"]

In [5]:
qt = QuantileTransformer(output_distribution="normal")
df[features] = qt.fit_transform(df[features])

In [6]:
inds = pd.read_csv("siccodes12.csv", index_col="industry")
ind_names = inds.index.unique().to_list()

def industry(sic):
  try:
    return inds[(inds.start<=sic)&(sic<=inds.end)].index[0]
  except:
    return "Other"
    
df["industry"] = df.siccd.map(industry)
features.append("industry")

In [7]:
df["predict"] = pipe.predict(df[features])

# Additional imports

In [ ]:
from datetime import datetime
from pathlib import Path
import numpy as np

from alpaca.trading.client import TradingClient
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockLatestQuoteRequest
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

KEY = ""
SECRET_KEY = "" 

# Tradable and shortable

In [ ]:
trading_client = TradingClient(KEY, SECRET_KEY, paper=True)
assets = trading_client.get_all_assets()

df["tradable"] = {
  x.symbol: x.tradable for x in assets
}

df["shortable"] = {
  x.symbol: x.shortable for x in assets
}

# Bid and ask prices

In [ ]:
data_client = StockHistoricalDataClient(KEY, SECRET_KEY)
params = StockLatestQuoteRequest(
  symbol_or_symbols=df.index.to_list()
)
quotes = data_client.get_stock_latest_quote(params)

df["ask"] = {x: quotes[x].ask_price for x in quotes}
df["bid"] = {x: quotes[x].bid_price for x in quotes}

# Equity and current positions

In [ ]:
account = trading_client.get_account()
equity = float(account.equity)

current = trading_client.get_all_positions()
df["current"] = (
    {x.symbol: int(x.qty) for x in current} 
    if len(current)>0 else 0
)
df["current"] = df.current.fillna(0)

# Ranking tradable and shortable stocks

In [ ]:
df["predict"] = np.where(
  df.tradable & (df.ask>0), 
  df.predict, np.nan
)
df["rank_from_top"] = df.predict.rank(
  ascending=False, 
  method="first"
)

df["predict"] = np.where(
  df.shortable & (df.bid>0), 
  df.predict, np.nan
)
df["rank_from_bottom"] = df.predict.rank(method="first")

# Longs and shorts

In [ ]:
numlong = numshort = 100

df["long"] = df.rank_from_top <= numlong
df["short"] = df.rank_from_bottom <= numshort

print(f"number long is {len(df.long)}")
print(f"number short is {len(df.short)}")

# Targets and trades

In [ ]:
df["target_dollars"] = (
  (1.5*equity/numlong)*df.long - 
      (0.5*equity/numshort)*df.short
)
df["current_dollars"] = df.current*(df.bid+df.ask)/2
df["trade_dollars"] = df.target_dollars - df.current_dollars
df["trade"] = (
    (df.trade_dollars>0)*df.trade_dollars/df.ask
    + (df.trade_dollars<0)*df.trade_dollars/df.bid
)
df["trade"] = df.trade.fillna(0)
df["trade"] = df.trade.astype(int)

# Submit trades

In [ ]:
for tick in df[df.trade!=0].index:
  market_order_data = MarketOrderRequest(
    symbol=tick,
    qty=abs(df.trade[tick]),
    side=OrderSide.BUY if df.trade>0 else OrderSide.SELL,
    time_in_force=TimeInForce.DAY
  )
  market_order = trading_client.submit_order(
    order_data=market_order_data
  )

# Save data

In [ ]:
today = datetime.today().strftime("%Y-%m-%d")
df["date"] = today
df = df[["date", "trade"]].reset_index()

df["date"] = today

if Path('./trades.csv').is_file():
    d = pd.read_csv("trades.csv")
    d = d[d.date != today]
    d = pd.concat((d, df))
    df.to_csv("trades.csv")
else:
    df.to_csv("trades.csv")

account = trading_client.get_account()
account = dict(account)
account = pd.DataFrame(pd.Series(account)).T
account["date"] = today

if Path('./account.csv').is_file():
    d = pd.read_csv("account.csv")
    d = d[d.date != today]
    d = pd.concat((d, account))
    d.to_csv("account.csv")
else:
    account.to_csv("account.csv")

positions = trading_client.get_all_positions()
positions = {x.symbol: x.qty for x in positions}
positions = pd.DataFrame(pd.Series(positions))
positions["date"] = today

if Path('./positions.csv').is_file():
    d = pd.read_csv("positions.csv")
    d = d[d.date != today]
    d = pd.concat((d, positions))
    d.to_csv("positions.csv")
else:
    positions.to_csv("positions.csv")
